In [1]:
from google.colab.auth import authenticate_user
authenticate_user()

import google.cloud.bigquery.magics
google.cloud.bigquery.magics.context.project = 'your_project_id'

! gcloud config set project { google.cloud.bigquery.magics.context.project }

Updated property [core/project].


# COVID19国内感染者数オープンデータ

厚生労働省の公開する、新型コロナウイルス感染症オープンデータ ( https://www.mhlw.go.jp/stf/covid-19/open-data.html )を用いて集計を行います。

- covid19_jp : 新規陽性者数の日別推移 を取り込んだテーブルです。
  - Date 日付
  - ALL 全国陽性者数
  - Hokkaido 北海道の陽性者数
  - ︙
  - Okinawa 沖縄の陽性者数
- covid19_jp_unpivotted 上のテーブルをunpivotしたテーブルです。ただし、0人の行は削除しています。
  - date 日付
  - prefecture 都道府県名（ローマ字）
  - new_cases 陽性者数

また、次のテーブルを使用することができます
- japan_population: 日本の都道府県別人口（令和二年国勢調査）
  - prefecture 都道府県名（ローマ字）
  - population 人口

## 問1
好きな日付（2020年9月〜2023年3月、以後省略）で新規感染者数が多い順に３つ都道府県を挙げてみましょう。

### 期待する出力フォーマット

|prefecture| new_cases|
|---:|---:|
|Tokyo| 12345|
|Osaka| 9876|
|Kanagawa| 5432|

In [2]:
%%bigquery

SELECT
  prefecture, new_cases
FROM `db_tutorial_2023.covid19_jp_unpivotted`
WHERE date = '2022-08-31'
ORDER BY new_cases desc LIMIT 3

Query is running:   0%|          |

Downloading:   0%|          |

,prefecture,new_cases
0,Tokyo,15428
1,Osaka,13678
2,Aichi,12612


In [4]:
%%bigquery

SELECT
  prefecture
FROM `db_tutorial_2023.covid19_jp_unpivotted`
WHERE date = '2022-08-31'
ORDER BY new_cases desc LIMIT 3 -- ソートする列をSELECTに必ずしも書かずともよい

Query is running:   0%|          |

Downloading:   0%|          |

,prefecture
0,Tokyo
1,Osaka
2,Aichi


## 問2

好きな日付で、人口あたりの新規感染者数が多い順に３つ都道府県を挙げてみましょう

Tips: 最近のアップデートでカラム名に一部のマルチバイト文字が対応したので、試してみてもよいでしょう（まだPreview段階の機能なので注意）。
https://cloud.google.com/bigquery/docs/release-notes#March_22_2023

### 期待する出力フォーマット

|prefecture| 百万人あたりの感染者数|
|---:|---:|
|Tokushima| 789|
|Fukuoka| 567|
|Iwate| 321|

In [5]:
%%bigquery
SELECT prefecture, new_cases/population * 1000000 `百万人あたりの感染者数`
FROM `db_tutorial_2023.covid19_jp_unpivotted`
JOIN `db_tutorial_2023.japan_population` using (prefecture)
WHERE date = '2022-08-31'
ORDER BY 2 desc -- 列名を書かずとも、2列目をこのように指定できる
LIMIT 3

Query is running:   0%|          |

Downloading:   0%|          |

,prefecture,百万人あたりの感染者数
0,Tokushima,2415.368302
1,Nagasaki,2202.211813
2,Miyazaki,2135.425627


In [6]:
%%bigquery
-- ON句で結合する場合
SELECT c.prefecture, new_cases/population * 1000000 `百万人あたりの感染者数`
FROM `db_tutorial_2023.covid19_jp_unpivotted` c
JOIN `db_tutorial_2023.japan_population` p ON c.prefecture = p.prefecture
WHERE date = '2022-08-31'
ORDER BY 2 desc LIMIT 3

Query is running:   0%|          |

Downloading:   0%|          |

,prefecture,百万人あたりの感染者数
0,Tokushima,2415.368302
1,Nagasaki,2202.211813
2,Miyazaki,2135.425627


## 問3

2021年4月1日〜30日において、新規感染者数が100人を超えていた都道府県はいくつあったか、日付別に出力しましょう。

### 期待する出力フォーマット

|date| pref_num|
|---:|---:|
|2021-04-01| 7|
|2021-04-02| 9|
|︙|︙|


In [7]:
%%bigquery

SELECT date, COUNT(1) pref_num
FROM `db_tutorial_2023.covid19_jp_unpivotted` c
WHERE
  date BETWEEN '2021-04-01' AND '2021-04-30' -- または、 DATE_TRUNC(date, MONTH) = '2021-04-01' など
  AND new_cases > 100
GROUP BY date ORDER BY date

Query is running:   0%|          |

Downloading:   0%|          |

,date,pref_num
0,2021-04-01,7
1,2021-04-02,9
2,2021-04-03,8
3,2021-04-04,7
4,2021-04-05,2
5,2021-04-06,6
6,2021-04-07,8
7,2021-04-08,7
8,2021-04-09,9
9,2021-04-10,9


## 問4
好きな日付・都道府県を選びます。

その都道府県が、その日何番目に新規感染者数が多い都道府県だったか、調べてみましょう。

出力は1レコードにしてください。

ヒント: https://cloud.google.com/bigquery/docs/reference/standard-sql/numbering_functions?hl=ja#rank

### 期待する出力フォーマット

|prefecture| new_cases| rank_number|
|---:|---:|---:|
|Ishikawa| 2085| 27|


In [8]:
%%bigquery
-- ありがちなミス
-- date = '2022-08-31' AND prefecture = 'Ishikawa' で絞ってから、RANK関数が実行されるので誤り。

SELECT prefecture, new_cases, RANK() OVER (PARTITION BY date ORDER BY new_cases DESC) rank_number
FROM `db_tutorial_2023.covid19_jp_unpivotted`
WHERE date = '2022-08-31' AND prefecture = 'Ishikawa'

Query is running:   0%|          |

Downloading:   0%|          |

,prefecture,new_cases,rank_number
0,Ishikawa,2085,1


In [9]:
%%bigquery
-- 想定回答
SELECT * FROM (
  SELECT prefecture, new_cases, RANK() OVER (PARTITION BY date ORDER BY new_cases DESC) rank_number
  FROM `db_tutorial_2023.covid19_jp_unpivotted`
  WHERE date = '2022-08-31'
) WHERE prefecture = 'Ishikawa'

Query is running:   0%|          |

Downloading:   0%|          |

,prefecture,new_cases,rank_number
0,Ishikawa,2085,27


In [10]:
%%bigquery
-- BigQuery特有のスマートな書き方
SELECT prefecture, new_cases, RANK() OVER (PARTITION BY date ORDER BY new_cases DESC) rank_number
FROM `db_tutorial_2023.covid19_jp_unpivotted`
WHERE date = '2022-08-31'
QUALIFY prefecture = 'Ishikawa'

Query is running:   0%|          |

Downloading:   0%|          |

,prefecture,new_cases,rank_number
0,Ishikawa,2085,27


## 問5

全期間（2020年1月〜）で、初めて感染者が出た日が一番遅かった都道府県はどこか、またその日付を調べてみましょう。

### 期待する出力フォーマット

|prefecture| first_date|
|---:|---:|
|Xxxxx| 2020-xx-xx|

In [14]:
%%bigquery
select * from (
SELECT prefecture, MIN(date) first_date
FROM `db_tutorial_2023.covid19_jp_unpivotted`
WHERE new_cases >= 1
GROUP BY 1
)
qualify RANK() OVER(ORDER BY first_date DESC)   = 1

Query is running:   0%|          |

Downloading:   0%|          |

,prefecture,first_date
0,Iwate,2020-07-29


## 問6

好きな1ヶ月を選びます。各日付において、その日3番目に感染者数が多い県を調べてみましょう。

### 期待する出力フォーマット

|date| pref_num|
|---:|---:|
|2022-10-01| Kanagawa|
|2022-10-02| Kanagawa|
|︙|︙|

In [16]:
%%bigquery
SELECT date, prefecture
FROM `db_tutorial_2023.covid19_jp_unpivotted`
WHERE DATE_TRUNC(date, MONTH) = '2022-10-01'
QUALIFY RANK() OVER(PARTITION BY date ORDER BY new_cases desc) = 3
ORDER BY date

Query is running:   0%|          |

Downloading:   0%|          |

,date,prefecture
0,2022-10-01,Kanagawa
1,2022-10-02,Kanagawa
2,2022-10-03,Osaka
3,2022-10-04,Hokkaido
4,2022-10-05,Hokkaido
5,2022-10-06,Kanagawa
6,2022-10-07,Osaka
7,2022-10-08,Hokkaido
8,2022-10-09,Kanagawa
9,2022-10-10,Kanagawa


## 問7

好きな都道府県の1ヶ月分の新規感染者数に、昨日より増えたかを表す列を追加してみましょう。

### 期待する出力フォーマット

|date| new_cases| is_incresed|
|---:|---:|---:|
|2023-01-01| 1378| False|
|2023-01-02| 1174| False|
|2023-01-03| 1318| True|


In [17]:
%%bigquery
SELECT date, new_cases, LAG(new_cases) OVER (order by date) < new_cases is_incresed
FROM (
  SELECT date, Nagano as new_cases -- 列名を変えたいだけ
  FROM `db_tutorial_2023.covid19_jp`
)
WHERE date between '2022-12-31' AND '2023-01-31' -- 前日の値を参照するので、一日前も見る
QUALIFY date between '2023-01-01' AND '2023-01-31' -- 1ヶ月分に絞っておく
ORDER BY 1

Query is running:   0%|          |

Downloading:   0%|          |

,date,new_cases,is_incresed
0,2023-01-01,1378,False
1,2023-01-02,1174,False
2,2023-01-03,1318,True
3,2023-01-04,1493,True
4,2023-01-05,3164,True
5,2023-01-06,3989,True
6,2023-01-07,3202,False
7,2023-01-08,3182,False
8,2023-01-09,1419,False
9,2023-01-10,1311,False


## 問8 (少し難しい)

前問で作成した結果を用いて、新規感染者数前日比増の連続日数（例: 3日連続増加）の、その期間での最大値を集計してください。

### 期待する出力フォーマット

| streak|
|---:|
|4|


### ヒント:
- Trueが連続する区間にラベル付をして、同じラベルがいくつあるか数えます。
- 連続区間のラベルは、True/Falseを区別しない通し番号と、Trueのときだけ増える番号の差を使います。


| 日付 | True/False | 通し番号 | True番号 | 差 |
|---:|---:|---:|---:|---:|
|2023-01-01|True|1|1| 0|
|2023-01-02|False|2|||
|2023-01-03|True|3|2|1|
|2023-01-04|True|4|3|1|
|2023-01-05|True|5|4|1|
|2023-01-06|False|6|

In [18]:
%%bigquery

WITH t AS (
  SELECT date, new_cases, LAG(new_cases) OVER (order by date) < new_cases flag
  FROM (
    SELECT date, Nagano as new_cases
    FROM `db_tutorial_2023.covid19_jp`
  )
  WHERE date between '2022-12-31' AND '2023-01-31'
  QUALIFY date between '2023-01-01' AND '2023-01-31'
)
-- 中間テーブルの様子
SELECT
  date,
  flag,
  ROW_NUMBER() OVER (ORDER BY date) rn,
  CASE
    WHEN flag THEN
    ROW_NUMBER() OVER (PARTITION BY flag ORDER BY date)
  END rn_true,
  CASE
    WHEN flag THEN
    ROW_NUMBER() OVER (ORDER BY date) - ROW_NUMBER() OVER (PARTITION BY flag ORDER BY date)
  END group_number
FROM t
order by date

Query is running:   0%|          |

Downloading:   0%|          |

,date,flag,rn,rn_true,group_number
0,2023-01-01,False,1,<NA>,<NA>
1,2023-01-02,False,2,<NA>,<NA>
2,2023-01-03,True,3,1,2
3,2023-01-04,True,4,2,2
4,2023-01-05,True,5,3,2
5,2023-01-06,True,6,4,2
6,2023-01-07,False,7,<NA>,<NA>
7,2023-01-08,False,8,<NA>,<NA>
8,2023-01-09,False,9,<NA>,<NA>
9,2023-01-10,False,10,<NA>,<NA>


In [19]:
%%bigquery
WITH t AS (
  SELECT date, new_cases, LAG(new_cases) OVER (order by date) < new_cases flag
  FROM (
    SELECT date, Nagano as new_cases
    FROM `db_tutorial_2023.covid19_jp`
  )
  WHERE date between '2022-12-31' AND '2023-01-31'
  QUALIFY date between '2023-01-01' AND '2023-01-31'
),
t2 AS (
  SELECT
    date,
    flag,
    ROW_NUMBER() OVER (ORDER BY date) rn,
    CASE
      WHEN flag THEN
      ROW_NUMBER() OVER (PARTITION BY flag ORDER BY date)
    END rn_true,
    CASE
      WHEN flag THEN
      ROW_NUMBER() OVER (ORDER BY date) - ROW_NUMBER() OVER (PARTITION BY flag ORDER BY date)
    END group_number
  FROM t
)
SELECT group_number, COUNT(1) streak
FROM t2
WHERE group_number IS NOT NULL
GROUP BY group_number ORDER BY streak desc limit 1


Query is running:   0%|          |

Downloading:   0%|          |

,group_number,streak
0,2,4


## 問9

2021年2月14日、新規感染者数が**少ない順**に10位（タイ含む）までを挙げてみましょう

この日は新規感染者数が0の県があることに注意して下さい


In [20]:
%%bigquery
WITH t AS (
  SELECT prefecture, SUM(new_cases) new_cases
  FROM (
    SELECT prefecture, new_cases
    FROM `db_tutorial_2023.covid19_jp_unpivotted`
    WHERE date = '2021-02-14'
    UNION ALL
    SELECT DISTINCT prefecture, 0
    FROM `db_tutorial_2023.covid19_jp_unpivotted`
  )
  GROUP BY 1
),
t2 AS ( -- tの別解
    SELECT prefecture, IFNULL(new_cases, 0) new_cases
    FROM (
      SELECT DISTINCT prefecture
      FROM `db_tutorial_2023.covid19_jp_unpivotted`
    )
    LEFT JOIN (
      SELECT prefecture, new_cases
      FROM `db_tutorial_2023.covid19_jp_unpivotted`
      WHERE date = '2021-02-14'
    ) USING (prefecture)
)

SELECT prefecture, new_cases, rank() over(order by new_cases) rn
FROM t2
QUALIFY rn <= 10
ORDER BY rn

Query is running:   0%|          |

Downloading:   0%|          |

,prefecture,new_cases,rn
0,Saga,0,1
1,Nagasaki,0,1
2,Shimane,0,1
3,Nagano,0,1
4,Ehime,0,1
5,Fukui,0,1
6,Tottori,0,1
7,Akita,0,1
8,Iwate,1,9
9,Toyama,1,9


## 問10

2021年、毎日（365日とも）新規感染者数が1以上だった都道府県を調べてみましょう。


In [21]:
%%bigquery
SELECT prefecture, count(1) cnt
from `db_tutorial_2023.covid19_jp_unpivotted`
WHERE  date between '2021-01-01' AND '2021-12-31' AND new_cases >= 1
GROUP BY prefecture
HAVING cnt = 365 -- COUNT(1) = 365 と書いても良い

Query is running:   0%|          |

Downloading:   0%|          |

,prefecture,cnt
0,Hokkaido,365
1,Tokyo,365
2,Kanagawa,365
3,Osaka,365


## 問11
北海道、東京都、神奈川県、大阪府**以外**の好きな府県を選んでください。

2021年、1番新規感染者数が多かった7日間、少なかった7日間を調べてみましょう。

(1) covid19_jpを用いて書いてみましょう。

(2) covid19_jp_unpivotted を用いて書いてみましょう。

### 期待する出力フォーマット

| start_date| end_date| sum_7days|
|---:|---:|---:|
|2021-11-20| 2021-11-27| 0|
|2021-11-21| 2021-11-28| 0|
|2021-08-17| 2021-08-24| 888|

In [22]:
%%bigquery
WITH t AS (
  SELECT
    date,
    new_cases,
    SUM(new_cases) OVER (order by date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) sum_7days -- 6日前〜その日の合計
  FROM (
    SELECT date, Nagano as new_cases
    FROM `db_tutorial_2023.covid19_jp`
  )
  WHERE date between '2021-01-01' AND '2021-12-31'
)

SELECT
  date - 7 start_date,
  date end_date,
  sum_7days
FROM t
QUALIFY
-- 少なかった7日間が複数あった場合に対応するために誠実に書く
RANK() OVER(ORDER BY sum_7days DESC) = 1 OR RANK() OVER(ORDER BY sum_7days) = 1

Query is running:   0%|          |

Downloading:   0%|          |

,start_date,end_date,sum_7days
0,2021-11-22,2021-11-29,0
1,2021-11-19,2021-11-26,0
2,2021-11-21,2021-11-28,0
3,2021-11-20,2021-11-27,0
4,2021-11-11,2021-11-18,0
5,2021-08-17,2021-08-24,888


In [23]:
%%bigquery
-- 実務では値が0の行がないテーブルを扱うケースが圧倒的に多い（COUNT関数で集計した結果など）。あらかじめ0の行を作っておきます。
WITH
covid19_nagano_2021 AS (
  SELECT date, SUM(new_cases) new_cases
  FROM (
    SELECT date, new_cases
    FROM `db_tutorial_2023.covid19_jp_unpivotted`
    WHERE prefecture = 'Nagano' AND date between '2021-01-01' AND '2021-12-31'
    UNION ALL
    SELECT date, 0
    FROM UNNEST(GENERATE_DATE_ARRAY('2021-01-01', '2021-12-31')) date
  )
  GROUP BY date
),
t AS (
  SELECT
    date,
    new_cases,
    SUM(new_cases) OVER (order by date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) sum_7days -- 6日前〜その日の合計
  FROM covid19_nagano_2021
)
SELECT
  date - 7 start_date,
  date end_date,
  sum_7days
FROM t
QUALIFY RANK() OVER(ORDER BY sum_7days DESC) = 1 OR RANK() OVER(ORDER BY sum_7days) = 1

Query is running:   0%|          |

Downloading:   0%|          |

,start_date,end_date,sum_7days
0,2021-11-11,2021-11-18,0
1,2021-11-21,2021-11-28,0
2,2021-11-19,2021-11-26,0
3,2021-08-17,2021-08-24,888
4,2021-11-22,2021-11-29,0
5,2021-11-20,2021-11-27,0


## 問12

好きな一ヶ月を選びます。都道府県ごとの10万人あたりの感染者数の毎日の中央値を出してください。

### 期待するフォーマット
|date| median|
|---:| ---:|
|2020-01-01| 12|
|2020-01-02| 14|
|︙|︙|

In [25]:
%%bigquery
-- 誤答例。2023年を選べば正答例に一致しますが、2021年だとだめな日もあります。
--
-- PERCENTILE_CONTでよいでしょうか？ 0人の都道府県を含む日も考慮に入れましょう。
SELECT distinct date, median
FROM (
  select date, prefecture, new_cases, PERCENTILE_CONT(new_cases / population * 100000, 0.5) over(partition by date ) median
  FROM `db_tutorial_2023.covid19_jp_unpivotted`
  JOIN `db_tutorial_2023.japan_population` USING (prefecture)
  where date between '2021-01-01' and '2021-01-31'
) order by 1

Query is running:   0%|          |

Downloading:   0%|          |

,date,median
0,2021-01-01,1.303217
1,2021-01-02,1.128481
2,2021-01-03,1.301532
3,2021-01-04,1.661038
4,2021-01-05,1.872882
5,2021-01-06,2.394582
6,2021-01-07,3.250346
7,2021-01-08,2.957698
8,2021-01-09,2.767964
9,2021-01-10,2.509339


In [29]:
%%bigquery
-- 47都道府県の中央値は24番目なので、こういう方法もあります。
-- ただ、新規感染者が存在する都道府県の数が24未満の場合は成立しません。

select date, new_cases / population * 100000
FROM `db_tutorial_2023.covid19_jp_unpivotted`
JOIN `db_tutorial_2023.japan_population` using (prefecture)
where date between '2021-01-01' and '2021-01-31'
qualify row_number() over(partition by date order by new_cases / population desc )  = 24
order by 1

Query is running:   0%|          |

Downloading:   0%|          |

,date,f0_
0,2021-01-01,1.237681
1,2021-01-02,1.012253
2,2021-01-03,1.300695
3,2021-01-04,1.363771
4,2021-01-05,1.872882
5,2021-01-06,2.394582
6,2021-01-07,3.139160
7,2021-01-08,2.957698
8,2021-01-09,2.767964
9,2021-01-10,2.429030


In [32]:
%%bigquery

WITH t AS (
  SELECT -- 問11と同じく0の行を作り出してUNIONする。
    date,
    prefecture,
    0 as new_cases,
    population
  FROM
    UNNEST(GENERATE_DATE_ARRAY('2020-04-01', '2020-04-30')) date
  CROSS JOIN db_tutorial_2023.japan_population

  UNION ALL
  SELECT
    date,
    prefecture,
    new_cases,
    population
  FROM db_tutorial_2023.covid19_jp_unpivotted
  JOIN db_tutorial_2023.japan_population USING (prefecture)
  WHERE date BETWEEN '2020-04-01' AND '2020-04-30'
),
t2 AS (
  SELECT
    date,
    prefecture,
    population,
    SUM(new_cases) AS new_cases
  FROM t
  GROUP BY 1,2,3
)
select date, new_cases / population * 100000
FROM t2
qualify row_number() over(partition by date order by new_cases / population desc )  = 24
order by 1

Query is running:   0%|          |

Downloading:   0%|          |

,date,f0_
0,2020-04-01,0.093631
1,2020-04-02,0.088980
2,2020-04-03,0.151003
3,2020-04-04,0.088980
4,2020-04-05,0.074915
5,2020-04-06,0.088980
6,2020-04-07,0.163382
7,2020-04-08,0.191402
8,2020-04-09,0.161553
9,2020-04-10,0.185617
